In [ ]:
# Add a summary of big O notation and corresponding operations for below data structures... 

# • List

Mutable array structure for storing elements of the same or different type. Lots of operations available on lists, especially homogenious data types. 

Hit tab next to a list instance to see available operations. Be aware that list, tuples, and other data structures have additional built in logic such as max(), min(), len(), etc... 

In [39]:
example = ['abc', 1472, ['another', 'list'], 23, True]

In [40]:
for item in example: # item is an arbitrary name, use whatever makes the most sense in the context of the loop 
    print(item)

abc
1472
['another', 'list']
23
True


In [41]:
example.append('add this to the end')
print(example)
print(id(example))

['abc', 1472, ['another', 'list'], 23, True, 'add this to the end']
4548520328


In [42]:
# If adding two large lists together use extend, this adds the second list to an existing list 
# Better than doing list1 + list2 as this will cause a new list to be created and both list 1+2 to be copied to it
example.extend([0, 1, 2, 3, 4, 5, 6, 7, 8, 1472, 0]) 
print(example)
print(id(example))

['abc', 1472, ['another', 'list'], 23, True, 'add this to the end', 0, 1, 2, 3, 4, 5, 6, 7, 8, 1472, 0]
4548520328


In [43]:
print("The number 1472 appears {} times in the list".format(example.count(1472)))

The number 1472 appears 2 times in the list


In [44]:
example.pop(0) # Add index if desired
print(example)

[1472, ['another', 'list'], 23, True, 'add this to the end', 0, 1, 2, 3, 4, 5, 6, 7, 8, 1472, 0]


In [45]:
if 0 in example:
    example.remove(0) # only removes a single instance... use while to remove all
print(example)

[1472, ['another', 'list'], 23, True, 'add this to the end', 1, 2, 3, 4, 5, 6, 7, 8, 1472, 0]


In [46]:
example.insert(0, 7) # Insert into index 0 the number 7, shifts all items to the right one
print(example)
print("example now has {} items in it".format(len(example)))

[7, 1472, ['another', 'list'], 23, True, 'add this to the end', 1, 2, 3, 4, 5, 6, 7, 8, 1472, 0]
example now has 16 items in it


In [47]:
example.clear()
print(example)

[]


In [48]:
nums = [1,5,7,233,23,12,78,45]
print(max(nums))

233


# • Deque
A list like container with fast appends and pops on either end... Notice speed difference vs list for ordered updates. This is due to the fact a list moves every item when inserting to the front O(N) where as deque just changes the pointer O(1). 

Adding or popping items from either end of deque has O(1) complexity.    
Adding or popping items from the front of a list is O(N), adding to the end of the list is O(1).

In [49]:
nums = [x for x in range(20000)]

In [50]:
%%timeit
nums.pop(0)
nums.insert(0, 10)

8.3 µs ± 285 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


In [51]:
from collections import deque
nums = deque(x for x in range(20000))

In [52]:
%%timeit
nums.popleft()
nums.appendleft(10)

90.2 ns ± 0.562 ns per loop (mean ± std. dev. of 7 runs, 10000000 loops each)


# • Tuples 

Immutable structure also able to handle multiple data types. Tuples have a smaller memory footprint vs lists.       
Being immutable, tuples are space allocated on the number of assignments at creation, no more.    
Lists and other mutable structures like dictionaries have extra space allocated to allow for adding more data to the structure.     
Depending on the number of elements this can be a fair amount of extra space vs a tuple.

In [53]:
tuple_ = () # Empty tuple
single_element_tuple = (1,) # Comma Needed when only one item present, otherwise declared an int with value 1

tup = ('green', True, 'Howdy!', 11)
print(type(tup))

<class 'tuple'>


In [54]:
color, bool_, greeting, answer = tup
print(color, bool_, greeting, answer)

green True Howdy! 11


In [55]:
print("Count of 'Howdy!' == {}".format(tup.count('Howdy!')))

Count of 'Howdy!' == 1


In [56]:
print(tup.index(11))

3


# • Named tuples

Tuple with naming additions to make code more readable. Still keeps a low memory profile like normal tuples. 

In [57]:
from collections import namedtuple

In [58]:
EmployeeRecord = namedtuple('Employee', 'name, age, title, department, pay')
rawdata = "Chris 49 Engineer Design 190000".split()
example = EmployeeRecord(*rawdata)

print("Age : {}".format(example.age))
print(example)
from sys import getsizeof
print(getsizeof(example))
print(getsizeof(rawdata))

Age : 49
Employee(name='Chris', age='49', title='Engineer', department='Design', pay='190000')
88
160


In [59]:
# Can also treat like a regular tuple...
for entry in example:
    print(entry)

Chris
49
Engineer
Design
190000


# • Dictionaries 

Workhorse of python... Key value pairs with O(1) adds and retrievals     
Lots of things in python make heavy usage of dictionaries including classes 

In [60]:
dict1 = dict(A=1, Z=-1)
dict2 = {'A': 1, 'Z': -1}
dict3 = dict(zip(['A', 'Z'], [1, -1]))
dict4 = dict([('A', 1), ('Z', -1)])
dict5 = dict({'Z': -1, 'A': 1})

print(dict1 == dict2 == dict3 == dict4 == dict5)

True


In [61]:
names = ['Isabella', 'Nathan', 'Ranik', 'Sara']
nationalities = ['Belgium', 'British', 'Swiss', 'Nepal'] 
dict_ = dict(zip(names, nationalities))
print(dict_)
print(dict_.keys())
print(dict_.values())
print(dict_.items()) # Also the option of iteritems(), a generator equivalent 

{'Isabella': 'Belgium', 'Nathan': 'British', 'Ranik': 'Swiss', 'Sara': 'Nepal'}
dict_keys(['Isabella', 'Nathan', 'Ranik', 'Sara'])
dict_values(['Belgium', 'British', 'Swiss', 'Nepal'])
dict_items([('Isabella', 'Belgium'), ('Nathan', 'British'), ('Ranik', 'Swiss'), ('Sara', 'Nepal')])


In [62]:
# Can check if something in keys, values, or items
if 'Nathan' in dict_.keys():
    print(True)

True


In [63]:
# Operations that allow setting or getting default values without receiving key value errors
value = dict_.pop('thisisnotakey', 'return this instead')
print(value)
print(dict_)

return this instead
{'Isabella': 'Belgium', 'Nathan': 'British', 'Ranik': 'Swiss', 'Sara': 'Nepal'}


In [64]:
# Get value if exist, but don't modify the dictionary 
value = dict_.get('thisisnotakey', 'return this instead')
print(value)
print(dict_)

return this instead
{'Isabella': 'Belgium', 'Nathan': 'British', 'Ranik': 'Swiss', 'Sara': 'Nepal'}


In [65]:
# Use setdefault to check for a key and add it to the dictionary if not
value = dict_.setdefault('nokey', 'but add it as well')
print(value)
print(dict_)

but add it as well
{'Isabella': 'Belgium', 'Nathan': 'British', 'Ranik': 'Swiss', 'Sara': 'Nepal', 'nokey': 'but add it as well'}


In [66]:
# Can do logical operations on keys, values, items 
a = {'a' : 1, 'b' : 2, 'c' : 3}
b = {'b' : 2, 'c' : 4, 'd' : 6}

print('Common keys:', a.keys() & b.keys())
print('Keys from A not in B:', a.keys() - b.keys())
print('Key, value  pairs in common:', a.items() & b.items())

# Make a new dictionary with certain keys removed
c = {key:a[key] for key in a.keys() - {'z', 'w'}}
# c is {'x': 1, 'y': 2}

Common keys: {'c', 'b'}
Keys from A not in B: {'a'}
Key, value  pairs in common: {('b', 2)}


In [67]:
# Merge two dictionaries... if a key exist multiple times only the first value will be saved
a = {'a' : 1, 'b' : 2, 'c' : 3}
b = {'b' : 2, 'c' : 4, 'd' : 6}

from collections import ChainMap
c = ChainMap(a,b)
print(c['b'])      # Outputs 1  From A
print(c['c'])      # Outputs 2  From A
print(c['d'])      # Outputs 3  From B
print(list(c.values()))

2
3
6
[2, 3, 6, 1]


In [68]:
status = {"1z42244": "Green", '1z42245': "Blue"} #dictionary of already known status codes

def lookup(id_):
    return "Some Value"

status_id = "2a11452"
# Check a dictionary for a cached status, if not look it up and add it to the cache 
result = status.setdefault(status_id, lookup(status_id))  
print(result)
print("Status '{}' translates to '{}'".format(status_id, status[status_id]))
print(status)

Some Value
Status '2a11452' translates to 'Some Value'
{'1z42244': 'Green', '1z42245': 'Blue', '2a11452': 'Some Value'}


In [70]:
dict_ = {'_source' : {'results': 'all the data', 'people': 'All the people'}}
# Checking multiple layers deep or return 'Not in dictionary'... 
# Elasticsearch gold for avoiding key value errors...
people = dict_.get('_source', {}).get('people', 'Not in dictionary')
label = dict_.get('_source', {}).get('label', 'Not in dictionary')
print("People : ", people )
print("Label : ", label)

People :  All the people
Label :  Not in dictionary


In [71]:
people = ['Tomas', 'Julio', 'Mike', 'Mez']
ages = [[21, 'student'], [30, 'engineer'], [31, 'manager'], [30, 'artist']]
dict_ = dict(zip(people, ages))
print(dict_)

{'Tomas': [21, 'student'], 'Julio': [30, 'engineer'], 'Mike': [31, 'manager'], 'Mez': [30, 'artist']}


In [72]:
# Using '' as a default return value we can iterate on nested dictionary calls...
# Does not work if '' is replaced with a non iterable item such as None
for entity in dict_.get('Mez', ''):
    print(entity, end = ' ')

30 artist 

In [74]:
for entity in dict_.get('NotPresent', ''): # If '' is changed to None this will crash
    print(entity, end = ' ')

# • Heaps 

heapq is a heap for organizing min and max structures.     
O(log n) push and pop.     
O(n log n) to push all items on to the heap.     

In [75]:
import heapq

rows = [
    {'name': 'Steve', 'age': 19},
    {'name': 'John', 'age': 24},
    {'name': 'Sally', 'age': 32},
    {'name': 'Ada', 'age': 22}
]

top_three = heapq.nlargest(3, rows, key=lambda status: status['age'])

In [76]:
for item in top_three:
    print(item)

{'name': 'Sally', 'age': 32}
{'name': 'John', 'age': 24}
{'name': 'Ada', 'age': 22}


# • Collections module dictionary tools

In [77]:
# Default dictionary, set default for every new key 
from collections import defaultdict
ddict = defaultdict(int)  # int is the default type (0 the value)
ddict['year'] += 1
print(ddict['year'])

ddict['year'] = 1999
ddict['year'] += 1
print(ddict['year'])

1
2000


In [78]:
from collections import Counter 
words = "hello how are you doing".split()

counts = Counter(words)
print(counts)
print("Most Common:", counts.most_common(3))

Counter({'hello': 1, 'how': 1, 'are': 1, 'you': 1, 'doing': 1})
Most Common: [('hello', 1), ('how', 1), ('are', 1)]


In [79]:
more = "add more words here".split()

counts.update(more)
print(counts)
print("Counts on 'my':", counts['my'])

Counter({'hello': 1, 'how': 1, 'are': 1, 'you': 1, 'doing': 1, 'add': 1, 'more': 1, 'words': 1, 'here': 1})
Counts on 'my': 0


In [82]:
from collections import Counter
from collections import defaultdict
ddict = defaultdict(Counter)

ddict['nums'].update(['1', '2','3'])
ddict['test'].update(["this", "is", "nice"])
ddict['test'].update(["this", "is", "nice"])
ddict['test'].update(["this", "is", "nice"])
print(ddict)

defaultdict(<class 'collections.Counter'>, {'nums': Counter({'1': 1, '2': 1, '3': 1}), 'test': Counter({'this': 3, 'is': 3, 'nice': 3})})


### Other types of dictionries to explore... 

OrderedDict: dict subclass that remembers the order entries were added    
UserDict: A wrapper around dictionary objects for easier dict subclassing    
UserList: A wrapper around list objects for easier list subclassing    
UserString: A wrapper around string objects for easier string subclassing   

# • Set
The set type is mutable, while frozenset is immutable. They are unordered collections of immutable objects. 
Good for deduplicating values when temporary storing data for processing 

In [83]:
small = {1, 5, 6, 2}
large = set([6, 8, 9, 10, 8, 8])
print(small)
print(large)

{1, 2, 5, 6}
{8, 9, 10, 6}


In [84]:
all_ = small | large # union
intersection = small & large # intersection
difference = small - large # subtract out overlapping
print(all_)
print(intersection)
print(difference)

{1, 2, 5, 6, 8, 9, 10}
{6}
{1, 2, 5}
